## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-19-12-59-10 +0000,bbc,"VAT rate on energy bills could be cut, ministe...",https://www.bbc.com/news/articles/cdjrg4vylzko...
1,2025-10-19-12-55-00 +0000,wsj,Heist at Louvre Leaves Museum Missing Priceles...,https://www.wsj.com/world/europe/seven-minute-...
2,2025-10-19-12-53-56 +0000,cbc,Israeli military says it conducted airstrikes ...,https://www.cbc.ca/news/world/israel-military-...
3,2025-10-19-12-53-42 +0000,nyt,Gaza Cease-Fire Strained by New Flare-Up of Vi...,https://www.nytimes.com/2025/10/19/world/middl...
4,2025-10-19-12-46-39 +0000,bbc,"Israel launches air strikes in Gaza, accusing ...",https://www.bbc.com/news/articles/cm2elr16jjqo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2337/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,trump,29
57,no,18
58,kings,16
18,gaza,13
28,israel,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
14,2025-10-19-12-09-54 +0000,nyt,‘No Kings’ Protests Against Trump Draws Thousa...,https://www.nytimes.com/2025/10/18/us/protests...,90
139,2025-10-18-18-45-48 +0000,cbc,'Trump must go': Large crowds join 'No Kings' ...,https://www.cbc.ca/news/world/no-kings-protest...,88
80,2025-10-19-03-33-45 +0000,missionlocal,"Photos: Defiance, solidarity fill S.F. streets...",https://missionlocal.org/2025/10/photos-defian...,86
66,2025-10-19-06-11-20 +0000,bbc,Millions join anti-Trump 'No Kings' protests a...,https://www.bbc.com/news/articles/c93xgyp1zv4o...,86
7,2025-10-19-12-33-49 +0000,nypost,Trump posts wild AI video showing him flying f...,https://nypost.com/2025/10/19/us-news/trump-po...,80


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
14,90,2025-10-19-12-09-54 +0000,nyt,‘No Kings’ Protests Against Trump Draws Thousa...,https://www.nytimes.com/2025/10/18/us/protests...
4,50,2025-10-19-12-46-39 +0000,bbc,"Israel launches air strikes in Gaza, accusing ...",https://www.bbc.com/news/articles/cm2elr16jjqo...
127,33,2025-10-18-20-09-11 +0000,nypost,Zelensky’s drone attacks on oil refineries cau...,https://nypost.com/2025/10/18/world-news/zelen...
119,32,2025-10-18-21-08-36 +0000,nypost,Jeffrey Epstein sent nasty emails to billionai...,https://nypost.com/2025/10/18/us-news/jeffrey-...
137,29,2025-10-18-19-04-19 +0000,nypost,Obama-era Iran nuclear non-proliferation deal ...,https://nypost.com/2025/10/18/world-news/obama...
3,28,2025-10-19-12-53-42 +0000,nyt,Gaza Cease-Fire Strained by New Flare-Up of Vi...,https://www.nytimes.com/2025/10/19/world/middl...
128,26,2025-10-18-19-48-40 +0000,cbc,U.S. sending survivors of strike on alleged 'd...,https://www.cbc.ca/news/world/us-boat-strike-c...
37,25,2025-10-19-09-57-15 +0000,nypost,"Thieves break into Louvre, swipe priceless jew...",https://nypost.com/2025/10/19/world-news/louvr...
115,24,2025-10-18-22-04-02 +0000,nypost,NYC police union leader calls out Zohran Mamda...,https://nypost.com/2025/10/18/us-news/nyc-dete...
93,23,2025-10-19-00-44-00 +0000,48hills,Massive No Kings protest in SF—and all over th...,https://48hills.org/2025/10/massive-no-kings-p...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
